This notebook contains similarity preserving neural network training.

In [1]:
NUM_BITS = 128

In [2]:
from time import time
from pickle import load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras import backend as K

Using TensorFlow backend.


In [3]:
df = pd.read_csv('./dataset.csv', index_col=0)
df.head()

,color,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,...,Film-Noir,Family,Crime,Short,Biography,Animation,Horror,News,Thriller,Musical
Avatar,1.0,0.889163,0.529412,0.000000,0.037174,0.001563,1.000000,0.524453,0.007361,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: At World's End,1.0,0.370690,0.501548,0.024478,0.043478,0.062500,0.406840,0.278865,0.073622,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Spectre,1.0,0.740148,0.436533,0.000000,0.007000,0.017188,0.263080,0.163256,0.017816,0.023256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
The Dark Knight Rises,1.0,1.000000,0.486068,0.956522,1.000000,0.042188,0.589253,0.677216,0.162561,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
John Carter,1.0,0.567734,0.386997,0.020652,0.023043,0.001000,0.096066,0.125579,0.002852,0.023256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# load dict with similar/dissimilar movies
with open('./sims.pkl', 'rb') as f:
    movie_sims = load(f)

In [5]:
def create_pairs(movie_sims, df, split=0.85):
    """Creates positive/negative pairs for one-shot learning"""
    pairs = []
    labels = []

    for movie in movie_sims:
        # get vector for particular movie
        movie_vec = df.loc[movie].values
        # get vectors of its similar/dissimilar movies
        p_vec_l = [df.loc[movie].values for movie in movie_sims[movie]['pos']]
        n_vec_l =[df.loc[movie].values for movie in movie_sims[movie]['neg']]
        # construct pairs
        for pos, neg in zip(p_vec_l, n_vec_l):
            pairs += [[movie_vec, pos]]
            pairs += [[movie_vec, neg]]
            labels += [0, 1]
    
    # validation split
    split_indice = int(split * len(pairs))

    pairs_train = pairs[:split_indice]
    pairs_test = pairs[split_indice:]

    labels_train = labels[:split_indice]
    labels_test = labels[split_indice:]
    
    return ((np.array(pairs_train), np.array(labels_train)),
            (np.array(pairs_test), np.array(labels_test)))

In [6]:
def euclidean_distance(vects):
    """Compute euclidean distance."""
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1[0], 1


def margin_loss(y_true, y_pred):
    m = 1
    loss = 0.5*(1-y_true)*y_pred + 0.5*y_true*K.maximum(0.0, m - y_pred)
    return loss


def create_base_network(input_shape, num_bit=NUM_BITS):
    """Base network architecture."""
    input = Input(shape=input_shape)
    x = Dense(60, activation='relu')(input)
    x = Dropout(0.2)(x)
    x = Dense(80, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(num_bit, activation='sigmoid')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.'''
    pred = y_pred.ravel() > 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.'''
    return K.mean(K.equal(y_true, K.cast(y_pred > 0.5, y_true.dtype)))

In [7]:
    print('Creating pairs ...')
    (x_train, y_train), (x_test, y_test) = create_pairs(movie_sims, df)
    print('Pairs created!')
    x_test = x_test.astype('float32')
    input_shape = [x_train.shape[-1]]
    print('Input shape:', input_shape[0])

    base_network = create_base_network(input_shape)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    L1_distance = lambda x: K.abs(x[0] - x[1])

    distance = Lambda(L1_distance,
                      output_shape=lambda x: x[0])([processed_a, processed_b])

    pred = Dense(1, activation='sigmoid')(distance)
    model = Model(input=[input_a, input_b], outputs=[pred])

    epochs = 6
    rms = RMSprop()
    early_stopping = EarlyStopping(patience=2, restore_best_weights=True)
    model.compile(loss=[margin_loss], optimizer=rms, metrics=['accuracy'])
    model.fit([x_train[:, 0], x_train[:, 1]], y_train,
              batch_size=128,
              epochs=epochs,
              validation_data=([x_test[:, 0], x_test[:, 1]], y_test), callbacks=[early_stopping])

Creating pairs ...
Pairs created!
Input shape: 41


/home/kuba/anaconda3/envs/lsh_tf36/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`


Train on 388011 samples, validate on 68473 samples
Epoch 1/6
388011/388011 [==============================] - 42s 107us/step - loss: 0.0202 - acc: 0.9683 - val_loss: 0.0781 - val_acc: 0.9032
Epoch 2/6
388011/388011 [==============================] - 40s 104us/step - loss: 0.0038 - acc: 0.9939 - val_loss: 0.0348 - val_acc: 0.9649
Epoch 3/6
388011/388011 [==============================] - 26s 66us/step - loss: 0.0028 - acc: 0.9953 - val_loss: 0.0199 - val_acc: 0.9774
Epoch 4/6
388011/388011 [==============================] - 53s 135us/step - loss: 0.0021 - acc: 0.9964 - val_loss: 0.0154 - val_acc: 0.9830
Epoch 5/6
388011/388011 [==============================] - 42s 107us/step - loss: 0.0017 - acc: 0.9969 - val_loss: 0.0139 - val_acc: 0.9817
Epoch 6/6
388011/388011 [==============================] - 38s 97us/step - loss: 0.0015 - acc: 0.9974 - val_loss: 0.0114 - val_acc: 0.9855


In [8]:
y_pred = model.predict([x_test[:, 0], x_test[:, 1]])
te_acc = compute_accuracy(y_test, y_pred)
y_pred = model.predict([x_train[:, 0], x_train[:, 1]])
tr_acc = compute_accuracy(y_train, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 99.81%
* Accuracy on test set: 98.55%


In [9]:
# compute hash values for each movie
hashes = dict()
for movie in df.index.values:
    movie_v = df.loc[movie].values.reshape(1, 41)
    hash_value = base_network.predict(movie_v)
    # quantization
    hash_value[hash_value < 0.5] = 0
    hash_value[hash_value >= 0.5] = 1
    hashes[movie] = hash_value

In [10]:
# split hashes for DataFrame
hashes_split = dict()
for movie_name in hashes:
    hashes_split[movie_name] = [int(x) for x in hashes[movie_name][0]]

# load hash values as DataFrame
df_hashes = pd.DataFrame.from_dict(hashes_split, orient='index')
# save them to .csv
df_hashes.to_csv('./neural_hashes.csv', index=True)
df_hashes.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
Avatar,0,0,0,1,1,1,1,0,0,1,...,0,1,1,1,0,0,1,0,0,1
Pirates of the Caribbean: At World's End,0,0,0,1,1,1,1,0,0,1,...,0,1,1,1,0,1,1,0,0,1
Spectre,0,0,0,1,1,1,1,0,0,0,...,0,1,1,1,1,1,1,0,0,1
The Dark Knight Rises,0,0,0,1,1,1,1,0,0,0,...,0,1,1,1,1,1,1,0,0,1
John Carter,0,0,0,1,1,1,1,0,0,1,...,0,1,1,1,0,1,1,0,0,1


In [11]:
# compute cosine similarity for hashes
cos_h = cosine_similarity(df_hashes.values)
# compute cosine similarity DataFrame for hashes
df_sim_h = pd.DataFrame(cos_h, columns=df_hashes.index.values, index=df_hashes.index.values)
df_sim_h.head()

,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On the Downlow,Sanctuary; Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,The Mongol King,Newlyweds,Signed Sealed Delivered,My Date with Drew
Avatar,1.000000,0.961860,0.810443,0.674400,0.961860,0.917319,0.553912,0.961860,0.953959,0.961860,...,0.479702,0.762770,0.479702,0.477952,0.673056,0.502519,0.474781,0.417786,0.422028,0.893939
Pirates of the Caribbean: At World's End,0.961860,1.000000,0.816654,0.696143,1.000000,0.954655,0.527148,1.000000,0.945765,0.984615,...,0.483378,0.800641,0.483378,0.481615,0.710510,0.506370,0.513858,0.420988,0.425262,0.931325
Spectre,0.810443,0.816654,1.000000,0.845330,0.816654,0.788600,0.532535,0.816654,0.790734,0.832666,...,0.447214,0.866667,0.409946,0.537086,0.823558,0.544614,0.424184,0.438178,0.442627,0.762770
The Dark Knight Rises,0.674400,0.696143,0.845330,1.000000,0.696143,0.734651,0.451003,0.696143,0.651449,0.696143,...,0.482198,0.897085,0.482198,0.648593,0.939450,0.654654,0.477252,0.377964,0.381802,0.723747
John Carter,0.961860,1.000000,0.816654,0.696143,1.000000,0.954655,0.527148,1.000000,0.945765,0.984615,...,0.483378,0.800641,0.483378,0.481615,0.710510,0.506370,0.513858,0.420988,0.425262,0.931325


In [12]:
# example
df_sim_h['The Matrix'].nlargest(20)


The Matrix                            1.000000
Ender's Game                          0.992509
Terminator 3: Rise of the Machines    0.992395
The Matrix Revolutions                0.992395
Total Recall                          0.992395
Terminator 2: Judgment Day            0.992395
Battle Los Angeles                    0.992395
Walking Tall                          0.992395
Excessive Force                       0.992395
Code of Honor                         0.992395
The Terminator                        0.992395
The Matrix Reloaded                   0.984848
Batman & Robin                        0.984848
Dredd                                 0.984848
Hidalgo                               0.984848
Sucker Punch                          0.984848
The Man with the Iron Fists           0.984848
Diamond Ruff                          0.984848
Pride and Prejudice and Zombies       0.984732
Escape from New York                  0.984732
Name: The Matrix, dtype: float64